# Recap

* We have covered group experiments where the individuals had the same role.
* We created `Player` fields for the individual level data and `Group` fields 
* And as usual we created `Page` classes for each page we will see and add fields with `form_fields`. We created a template for each one.
* We created a `group` function to set payoffs and triggered it at the waiting page by setting `after_all_players_arrive` variable to the name of our function.
* Now we will see how the design changes when we have different roles.


## Group Experiments II - Heterogenous Groups

* Some technical aspects differ from the individual experiments:

| Aspect                     | Homogeneous Groups        | Heterogeneous Groups                                     |
| -----------------          | ------------------     | --------                                             |
| **Pages**                  | Everybody sees every page |  Some pages might be shown to specific groups only|
| **Variables / Calculations** | Calculations are the same for everybody   | Role specific calculations might be necessary |

## Heterogenous Groups
- In this section, we will focus on the case when the players have different roles. 
- Games with heterogeneous agents have some unique aspects we need to handle:

  - We should be able to identify each player in group in order to give them role-specific tasks object
  - We should be able to show each player different screens.


## Identifying Participants in a Group


* oTree already have a structure to deal with individuals roles in a group.
* `player.id_in_group`: Each player gets an integer from 1 to n as their id
* ![](images/id_in_group.png)

## Default behavior `id_in_group` in oTree {.smaller}

:::{.center}
![](images/mtc_grouping_with_ids.png){width="50%"}
:::

* `id_in_group` stays the same as for multiple rounds by default.
* When participants use the same link (as opposed to the individual links), they will be grouped one by one.
* First 2 participants will be in group 1, next 2 in group 2, etc.


## Defining Roles
- Roles can be defined based on `id_in_group` variable of the player.


:::{.center}
![](images/mtc_grouping_with_ids_roles.png){width="70%"}
:::


## Calling players when they all have the same role
* In the Public Goods Game, our players were identical. So we could loop over players like

```{.python code-line-numbers="false"}
# Gets all the players and puts them in a list
players = group.get_players()

for p in players:
  do_something(p)
  p.some_attribute = some_number
  #...
  
```

## Calling players by id: `group.get_player_by_id()`

* `get_player_by_id()` is a group method

* Takes a number from 1 to n to get the player object of the group member 

  ```{.python code-line-numbers="false"}
  trustor = group.get_player_by_id(1)
  trustee = group.get_player_by_id(2)
  
  trustor.payoff = endowment - sent_amount + returned amount
  
  ```

## Calling players by id: `group.get_player_by_id()`

* We can also add a condition in the loop

```{.python code-line-numbers="false"}
# Gets all the players and puts them in a list
players = group.get_players()

for p in players:

  if player.id_in_group == 1:
    do_something(p)
    p.some_attribute = some_number

  if player.id_in_group == 2:
    do_something_else(p)
    p.some_attribute = some_other_number
  
```

## Calling players by id: `group.get_player_by_id()`

* We can also add a condition in the loop

```{.python code-line-numbers="false"}
# Gets all the players and puts them in a list
players = group.get_players()

for p in players:

  if player.id_in_group == 1: # Trustor
    do_something(p)
    p.some_attribute = some_number

  if player.id_in_group == 2: # Trustee
    do_something_else(p)
    p.some_attribute = some_other_number
  
```

## Dislaying pages conditionally
* Built in method to be defined under a page (can be outside as well)

* If this method returns `True` for current player, the page is displayed. Otherwise, it will not.

* If there are any calculations triggered by page, they will be skipped as well.

    * For a specific player:

In [ ]:
class PageForTrustor(Page):

    def is_displayed(player):
        return player.id_in_group == 1


# Building a Public Goods Game

## Creating a new app
1. `otree startapp pgg`
2. Add it to `settings.py`.

```{.python filename="settings.py" code-line-numbers="false"}
# ...
SESSION_CONFIGS = [
    dict(
        name='pgg',
        app_sequence=['pgg'],
        num_demo_participants=3,
    ),
]
# ...
```

## Let's plan first our pages
| Page           | Description       |
| -------------- | ----------------- |
| `Contribution` | Contribute screen |
| `Results`      | Results screen    |

## We need a WaitPage before the calculations

| Page                  | Description       |
| --------------------- | ----------------- |
| `Contribution`        | Contribute screen |
| **`ResultsWaitPage`** | Waiting page      |
| `Results`             | Results screen    |


## Fields

| Variable       | Model         | Type              |                              |
| -------------- | ------------- | ----------------- | ---------------------------- |
| `contribution` | `Player`      | `CurrencyField()` | Player's contribution        |
| `ENDOWMENT`    | `C` Constants | Currency (`cu`)   | Initial endowment            |
| `MULTIPLIER`   | `C` C         | Integer           | Multiplier for contributions |

## Group Fields

- We should think of group fields as the variables that are the same for all players in the group.

- `total_contribution`
- `individual_share` (not strictly necessary but useful for a cleaner calculation)

## Fields

| Variable       | Model         | Type              |                              |
| -------------- | ------------- | ----------------- | ---------------------------- |
| `contribution` | `Player`      | `CurrencyField()` | Player's contribution        |
| `payoff`       | `Player`      | `CurrencyField()` | Player's payoff              |
| `ENDOWMENT`    | `C` Constants | Currency (`cu`)   | Initial endowment            |
| `MULTIPLIER`   | `C` C         | Integer           | Multiplier for contributions |
| `total_contribution` | `Group` | `CurrencyField()` | Total contribution of the group |
| `individual_share` | `Group` | `CurrencyField()` | Individual share of the group |